In [45]:
from time import sleep
import pandas as pandas
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
import pandas as pd
pd.set_option('display.max_rows', 200)


In [46]:
#we have to puth the path where the webdriver is located, inside the function
driver = webdriver.Chrome('/Users/admin/chromedriver')

In [61]:
def get_df_from_url(url, restaurant_name):
    driver.get(url)
    sleep(5)

    detailed_reviews_list=[]

    restaurant_header = driver.find_elements_by_css_selector('.photoHeader__373c0__YdvQE ')
    restaurant_average_rating = restaurant_header[0].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label').split()[0]
    restaurant_reviews_count = restaurant_header[0].find_element_by_css_selector('.text__373c0__2Kxyz').text.split()[0]
    try:
            exp = restaurant_header[0].find_elements_by_css_selector('.text__373c0__2Kxyz.text-bullet--after__373c0__3fS1Z.text-size--large__373c0__3t60B')[-1].text
            restaurant_expensiveness = len(exp)
    except:
            restaurant_expensiveness = 'N/C'

    while True:
        comment_blocs = driver.find_elements_by_css_selector('.review__373c0__13kpL.border-color--default__373c0__3-ifU')   
        
        for i in range(len(comment_blocs)):
            try:
                #we have to change this line as it is not robust
                date = comment_blocs[i].find_elements_by_css_selector('.arrange__373c0__2C9bH.gutter-1__373c0__2l5bx \
                .text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[0].text.split()[-1]

            except: 
                date = comment_blocs[i].find_elements_by_css_selector('.arrange__373c0__2C9bH.gutter-1__373c0__2l5bx \
                .text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[2].text.split()[-1]

            rating = comment_blocs[i].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label')[0]

            try: 
                username = comment_blocs[i].find_elements_by_css_selector('.user-passport-info .link-size--inherit__373c0__1VFlE')[0].text
            except:
                username = 'Not_Yelp_User'
            
            try:
                photos_for_review = comment_blocs[i].find_elements_by_css_selector('a.link__373c0__1G70M.link-size--inherit__373c0__1VFlE')[1].text[0]
            except:
                photos_for_review = 0
            
            try:
                user_location =  comment_blocs[i].find_elements_by_css_selector('.user-passport-info .text__373c0__2Kxyz')[1].text
            except:
                user_location = 'no_location'

            text_review = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz .raw__373c0__3rcx7')[0].text
            user_friends_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[0].text
            user_reviews_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[1].text

            try:
                user_total_image_posted = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[2].text
                                                                                    
            except:
                user_total_image_posted = 0

            is_real_review = 1
            url = url

            detailed_reviews_list.append([date, username, photos_for_review, rating, text_review, user_location, user_friends_count, user_reviews_count,                                  user_total_image_posted, restaurant_average_rating, restaurant_reviews_count, restaurant_expensiveness, restaurant_name, url, is_real_review])

            
            print(url, i, 'OK', )

        try:
            #navigate to the next page
            driver.find_elements_by_css_selector('.next-link')[0].click()
            sleep(3) 
        except:
            break

    df = pd.DataFrame(detailed_reviews_list,
                                    columns = ['date','username', 'photos_for_review', 'rating',
                                                'text_review', 'user_location', 'user_friends_count',
                                                'user_reviews_count', 'user_total_image_posted', 'restaurant_average_rating',
                                                'restaurant_reviews_count', 'restaurant_expensiveness', 'restaurant_name', 'url', 'is_real_review'])
    return df


In [ ]:
get_df_from_url('https://www.yelp.fr/biz/le-d%C3%A9panneur-pigalle-paris-3', 'depanneur pigalle')

In [72]:
#we get the list of links and names previously scraped
dataset = pd.read_csv('Scrapping_list_of_restaurants_in_paris/restaurants_dataset_only_with_reviews.csv', index_col=0)


In [11]:

df_list = [pd.DataFrame() for i in range(len(dataset))]
error_index_list = []
start = 0
end = len(dataset)


for i in range(start, end):
    try:
        df_list[i] = get_df_from_url(dataset['link_to_restaurant'][i], dataset['restaurant_name'][i])
    except:
        error_index_list.append((i, dataset['link_to_restaurant'][i], dataset['restaurant_name'][i]))
        print('restaurant number', i, 'seems to have a problem')
    print('restaurant number ', i, 'OK')

    if i % 50==0 and i != 0 :
        #we create backups each 50 restaurants scraped so that we won't lose data in case of errors in the code
        #also we can interupt and rerun it, we will just have to change the start value at the beginning of the code
        checkpoint = [df_list[k] for k in range(i-50+1, i+1)]
        pd.concat(checkpoint).to_csv('checkpoints/reviews_part'+ str(i) +'.csv')
        pd.DataFrame(error_index_list).to_csv('logfiles/logfile'+ str(i) +'.csv')
        print('checkpoint', i, 'successfully saved')

    if i == len(dataset):
        checkpoint = [df_list[k] for k in range(i-50+1, i+1)]
        pd.concat(checkpoint).to_csv('checkpoints/reviews_part'+ str(i) +'.csv')
        pd.DataFrame(error_index_list).to_csv('logfiles/logfile'+ str(i) +'.csv')


p.fr/biz/le-grain-de-riz-paris 17 OK
https://www.yelp.fr/biz/le-grain-de-riz-paris 18 OK
https://www.yelp.fr/biz/le-grain-de-riz-paris 19 OK
https://www.yelp.fr/biz/le-grain-de-riz-paris 0 OK
restaurant number  4956 OK
https://www.yelp.fr/biz/phokthin-paris 0 OK
restaurant number  4957 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 0 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 1 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 2 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 3 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 4 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 5 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 6 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 7 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 8 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 9 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 10 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 11 OK
https://www.yelp.fr/biz/l%C3%BC%C3%BCk-paris-3 12 OK
https://www.yelp.fr/biz/l%C3%BC

In [20]:
import glob

path = 'checkpoints/'
# regex that to identify all files with similar_name 
regex = "review*"
filenames = glob.glob(path+regex)
# list comprehension to import and concatenate all files
full_dataset = pd.concat((pd.read_csv(file, index_col = 0) for file in filenames))
full_dataset = full_dataset.drop_duplicates()
